# WS13: Profit vs Planet

<h1 style="position: absolute; display: flex; flex-grow: 0; flex-shrink: 0; flex-direction: row-reverse; top: 60px;right: 30px; margin: 0; border: 0">
    <style>
        .markdown {width:100%; position: relative}
        article { position: relative }
    </style>
    <img src="https://gitlab.tudelft.nl/mude/public/-/raw/main/tu-logo/TU_P1_full-color.png" style="width:100px" />
    <img src="https://gitlab.tudelft.nl/mude/public/-/raw/main/mude-logo/MUDE_Logo-small.png" style="width:100px" />
</h1>
<h2 style="height: 10px">
</h2>

*[CEGM1000 MUDE](http://mude.citg.tudelft.nl/): Week 2.5, Optimization. For: December 13, 2023*

## Overview

A civil engineering company wants to decide on the projects that they should do. Their objective is to minimize the environmental impact of their projects while making enough profit to keep the company running.

They have a portfolio of 6 possible projects to invest in, where A, B , and C are new infrastructure projects (so-called type 1), and D, E, F are refurbishment projects (so-called type 2).

The environmental impact of each project is given by $I_i$ where $i \in [1,(...),6]$ is the index of the project. $I_i=[90,45,78,123,48,60]$

<div style="background-color:#facb8e; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px; width: 95%"> <p>Please note that the values above have been changed in comparison with the original workshop that was provided in class; these updated values makes the optimization result more interesting.</p></div>

The profit of each project is given by $P_i$ where $i\in [1,(...),6]$ is the index of the project: $P_i=[120,65,99,110,33,99]$

<div style="background-color:#facb8e; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px; width: 95%"> <p>The values above have also been changed.</p></div>

The company wants to do 3 out of the 6 projects, therefore please formulate the mathematical program that allows solving the problem, also knowing that the projects of type 2 must be at least as many as the ones of type 1 and that the profit of all projects together must be greater or equal than $250$ ($\beta$)

<b>You are not allowed to use ChatGPT for this task otherwise you won’t learn ;)</b>

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px; width: 95%">
<p>
<b>Task 1: Writting the mathematical formulation</b>   

Write down every formulation and constrain that is relevant to solve this optimization problem.
</p>
</div>

<div style="background-color:#FAE99E; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px; width: 95%">
<p>
<b>Start solution.</b>

$$Min(Z) \sum_{i=1}^{6} x_i I_i $$

subject to

$$ \sum_{i=1}^{6} x_i = 3 $$
$$ \sum_{i=4}^{6} x_i \ge \sum_{i=1}^{3} x_i $$
$$ \sum_{i=1}^{6} x_i P_i \ge β $$
$$x\in(1,0), i\in(1,....,6)$$
    
Extra Challenge (Task 6)
$$ \sum_{i=1}^{6} x_i I_i \le x_1 γ + (1-x_1) M $$
    
where M is a sufficiently big number such as 9999    
    
</p>
</div>

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px; width: 95%">
<p>
<b>Task 2: Setting up the problem</b>   

Define any variables you might need to setup your model.
</p>
</div>

In [11]:
# Project data
I = [90, 45, 78, 123, 48, 60]  # Environmental impact
P = [120, 65, 99, 110, 33, 99]  # Profit

# Minimum required profit
beta = 250
M = 100000

# Number of projects and types
num_projects = len(I)
num_type1_projects = 3
num_type2_projects = num_projects - num_type1_projects

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px; width: 95%">
<p>
<b>Task 3: Setting up the problem</b>   

We'll continue using `gurobi` this week, which you've set up in last week's `PA12`. We'll use some other special packages as well. Therefor, create a new environment using `environment_MUDE_opt.yml` to continue today and on friday.

</p>
</div>

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px; width: 95%">
<p>
<b>Task 4: Create the Gurobi model</b>   

Create the Gurobi model, set your decision variables, your function and your constrains. Take a look at the book for an example implementation in Python if you don't know where to start.
</p>
</div>

In [12]:
import gurobipy as gp

In [13]:
# Create a Gurobi model
model = gp.Model("Project_Selection")

#You can always ask for help to understand a function of gurobi
#help(gurobipy.model.addVars)

# Decision variables
x = model.addVars(num_projects, vtype=gp.GRB.BINARY, name="x")

# Objective function: Minimize environmental impact
model.setObjective(sum(I[i] * x[i] for i in range(num_projects)), gp.GRB.MINIMIZE)

# Constraint: Select exactly 3 projects
model.addConstr(x.sum() == 3, "Select_Projects")

# Constraint: Number of type 2 projects must be at least as many as type 1 projects selected
model.addConstr(sum(x[i] for i in range(num_type2_projects, num_projects)) - sum(x[i] for i in range(num_type1_projects)) >= 0, "Type_Constraint")

# Constraint: Minimum profit requirement
model.addConstr(sum(P[i] * x[i] for i in range(num_projects)) >= beta, "Minimum_Profit")

# Optimize the model
model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3 rows, 6 columns and 18 nonzeros
Model fingerprint: 0xa9bc0e77
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+02]
Presolve time: 0.00s
Presolved: 3 rows, 6 columns, 17 nonzeros
Variable types: 0 continuous, 6 integer (6 binary)
Found heuristic solution: objective 228.0000000
Found heuristic solution: objective 198.0000000

Root relaxation: objective 1.855909e+02, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0       198.00000  198.00000  0.00%     -    0s

Explored 1 nodes (3 simplex iterations) in 0.01

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px; width: 95%">
<p>
<b>Task 5: Display your results</b>   

Display the model in a good way to interpret and print the solution of the optimization.
</p>
</div>

In [14]:
print("Model structure:")        
# see the model that you have built in a nice why to interpret
model.display()  

# Display the solution
if model.status == gp.GRB.OPTIMAL:
    print("Optimal Solution:")
    for i in range(num_projects):
        if x[i].x > 0.9:
            print(f"Project {i+1}: Selected")
else:
    print("No optimal solution found.")

    
print("Optimal Objective function Value", model.objVal)    

Model structure:
Minimize
  <gurobi.LinExpr: 90.0 x[0] + 45.0 x[1] + 78.0 x[2] + 123.0 x[3] + 48.0 x[4] + 60.0 x[5]>
Subject To
  Select_Projects: <gurobi.LinExpr: x[0] + x[1] + x[2] + x[3] + x[4] + x[5]> = 3
Type_Constraint: <gurobi.LinExpr: -1.0 x[0] + -1.0 x[1] + -1.0 x[2] + x[3] + x[4] +
 x[5]> >= 0
Minimum_Profit: <gurobi.LinExpr: 120.0 x[0] + 65.0 x[1] + 99.0 x[2] + 110.0 x[3] + 33.0
 x[4] + 99.0 x[5]> >= 250
Binaries
  ['x[0]', 'x[1]', 'x[2]', 'x[3]', 'x[4]', 'x[5]']
Optimal Solution:
Project 1: Selected
Project 5: Selected
Project 6: Selected
Optimal Objective function Value 198.0


<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px; width: 95%">
<p>
<b>Task 6: Additional constraint</b>   

Solve the model with an additional constraint: if project 1 is done then the impact of all projects together should be lower than $\gamma$ with $\gamma=130$.

> The value of $\gamma$ has been changed with respect to the original workshop to make the optimization more interesting

</p>
</div>

In [15]:
#Additional constraint
gamma = 130
model.addConstr(sum(I[i] * x[i] for i in range(num_projects)) <= gamma * x[0] + M * (1 - x[0]), "Impact_Constraint") 

<gurobi.Constr *Awaiting Model Update*>

In [16]:
# Optimize the model
model.optimize()

print("Model structure:")        
# see the model that you have built in a nice why to interpret
model.display()  

# Display the solution
if model.status == gp.GRB.OPTIMAL:
    print("Optimal Solution:")
    for i in range(num_projects):
        if x[i].x > 0.9:
            print(f"Project {i+1}: Selected")
else:
    print("No optimal solution found.")

    
print("Optimal Objective function Value", model.objVal)   

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 6 columns and 24 nonzeros
Model fingerprint: 0x37d09666
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [5e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint Impact_Constraint by 68.000000000

Presolve removed 4 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 228 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.280000000000e+02, best bound 2.280000000000e+02, gap 0.0000%
Model structure:
Minimize
  <gurobi.LinExpr: 90.

**End of notebook.**
<h2 style="height: 60px">
</h2>
<h3 style="position: absolute; display: flex; flex-grow: 0; flex-shrink: 0; flex-direction: row-reverse; bottom: 60px; right: 50px; margin: 0; border: 0">
    <style>
        .markdown {width:100%; position: relative}
        article { position: relative }
    </style>
    <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">
      <img alt="Creative Commons License" style="border-width:; width:88px; height:auto; padding-top:10px" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" />
    </a>
    <a rel="TU Delft" href="https://www.tudelft.nl/en/ceg">
      <img alt="TU Delft" style="border-width:0; width:100px; height:auto; padding-bottom:0px" src="https://gitlab.tudelft.nl/mude/public/-/raw/main/tu-logo/TU_P1_full-color.png"/>
    </a>
    <a rel="MUDE" href="http://mude.citg.tudelft.nl/">
      <img alt="MUDE" style="border-width:0; width:100px; height:auto; padding-bottom:0px" src="https://gitlab.tudelft.nl/mude/public/-/raw/main/mude-logo/MUDE_Logo-small.png"/>
    </a>
    
</h3>
<span style="font-size: 75%">
&copy; Copyright 2023 <a rel="MUDE Team" href="https://studiegids.tudelft.nl/a101_displayCourse.do?course_id=65595">MUDE Teaching Team</a> TU Delft. This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.